In [1]:
from collections import namedtuple
import pandas as pd
import roamability as rb
from pandas import DataFrame

### IMSI allocation

In [7]:
%%time
def print_check_result(df, db_name, table_name, imsi_range_start, imsi_range_end):
    print(f'Checking {db_name} {table_name} for: {imsi_range_start} - {imsi_range_end}')
    if df.empty:
        print('No items')
    else:
        display(df)

def check_dmi_imsi(imsi_range_start, imsi_range_end):
    sql_srt = '''
    SELECT SUBSTR(SI.IMSI_NUMBER,1,10) AS IMSI_RANGE, count(SI.IMSI_NUMBER) AS NUM
    FROM s_imsi si
    WHERE SI.IMSI_NUMBER BETWEEN {} AND {}
    GROUP BY SUBSTR(SI.IMSI_NUMBER,1,10)
    ORDER BY SUBSTR(SI.IMSI_NUMBER,1,10) desc
    '''.format(imsi_range_start, imsi_range_end)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 's_imsi', imsi_range_start, imsi_range_end)
    
    sql_srt = '''
    select SUBSTR(S.IMSI,1,10) AS IMSI_RANGE, count(S.IMSI) AS NUM
    from subscriber s
    where S.IMSI BETWEEN {} AND {}
    group by SUBSTR(S.IMSI,1,10)
    order by SUBSTR(S.IMSI,1,10) desc
    '''.format(imsi_range_start, imsi_range_end)
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'DMI', 'subscriber', imsi_range_start, imsi_range_end)
    
def check_ocs_imsi(imsi_range_start, imsi_range_end):
    sql_srt="""
    SELECT SUBSTRING(mi.imsi, 1, 12) AS IMSI_RANGE, count(mi.imsi) AS NUM
    FROM multi_imsi mi
    WHERE mi.imsi BETWEEN '{}' AND '{}'
    AND mi.end_date IS NULL
    GROUP BY SUBSTRING(mi.imsi, 1, 12)
    ORDER BY SUBSTRING(mi.imsi, 1, 12) DESC
    """.format(imsi_range_start, imsi_range_end)
    with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'OCS', 'multi_imsi', imsi_range_start, imsi_range_end)
    
    sql_srt="""
    SELECT SUBSTRING(si.imsi, 1, 12) AS IMSI_RANGE, count(si.imsi) AS NUM
    FROM SUBSCRIBER_IMSIS si
    WHERE si.imsi BETWEEN '{}' AND '{}'
    AND si.end_date IS NULL
    GROUP BY SUBSTRING(si.imsi, 1, 12)
    ORDER BY SUBSTRING(si.imsi, 1, 12) DESC
    """.format(imsi_range_start, imsi_range_end)
    with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
        df = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    print_check_result(df, 'OCS', 'SUBSCRIBER_IMSIS', imsi_range_start, imsi_range_end)

In [22]:
%%time
# Under Development

class DefinedSponsorRanges:
    
    def __init__(self, sponsor_name, imsi_range_esim, imsi_range_own_hlr, imsi_range_dmi_external_hlr, imsi_range_test,
                msisdn_range, rdb_sponsor_name):
        self.sponsor_name = sponsor_name
        self.imsi_range_esim = imsi_range_esim
        self.imsi_range_own_hlr = imsi_range_own_hlr
        self.imsi_range_dmi_external_hlr = imsi_range_dmi_external_hlr
        self.imsi_range_test = imsi_range_test
        self.msisdn_range = msisdn_range
        self.rdb_sponsor_name = rdb_sponsor_name

        
class SponsorRanges:

    def __init__(self, defined_sponsor_ranges, range_type):
        self.defined_sponsor_ranges = defined_sponsor_ranges
        self.range_type = range_type
        self.imsi_group_ranges = self.imsi_group_ranges()
        self.msisdn_group_ranges = self.msisdn_group_ranges()
    
    def define_range(self):
        if self.range_type == 'esim':
            return self.defined_sponsor_ranges.imsi_range_esim
        elif self.range_type == 'own_hlr':
            return self.defined_sponsor_ranges.imsi_range_own_hlr
        elif self.range_type == 'dmi_external_hlr':
            return self.defined_sponsor_ranges.imsi_range_dmi_external_hlr
        elif self.range_type == 'test':
            return self.defined_sponsor_ranges.imsi_range_test
        
    def fetch_imsi_data_from_rdb(self):
        
        sql_srt="""
        SELECT
        o.Name,i.RangeStart,i.RangeEnd,i.Allocation
        FROM ImsiRange i
        LEFT JOIN [dbo].[Sponsors] s ON i.SponsorID = s.ID
        LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
        WHERE o.Name = '{0}'
        AND i.RangeStart LIKE '{1}%';""".format(self.defined_sponsor_ranges.rdb_sponsor_name, self.define_range())
        with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
            df_rdb_imsi_range = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
        df_rdb_imsi_range.Name.replace(
            {self.defined_sponsor_ranges.rdb_sponsor_name:self.defined_sponsor_ranges.sponsor_name}, inplace=True)
        return df_rdb_imsi_range
    
    def fetch_msisdn_data_from_rdb(self):
        
        sql_srt="""
        SELECT
        o.Name,m.RangeStart,m.RangeEnd,m.Description
        FROM SponsorFakeMsisdnRanges m
        LEFT JOIN [dbo].[Sponsors] s ON m.SponsorID = s.ID
        LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
        WHERE o.Name = '{0}'
        AND m.RangeStart LIKE '{1}%';""".format(self.defined_sponsor_ranges.rdb_sponsor_name,
                                                self.defined_sponsor_ranges.msisdn_range)
        with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
            df_rdb_msisdn_range = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
        df_rdb_msisdn_range.Name.replace(
            {self.defined_sponsor_ranges.rdb_sponsor_name:self.defined_sponsor_ranges.sponsor_name}, inplace=True)
        df_rdb_msisdn_range[['RangeStart','RangeEnd']] = df_rdb_msisdn_range[['RangeStart','RangeEnd']].astype('int64')
        return df_rdb_msisdn_range
    
    def imsi_group_ranges(self):
        
        df_rdb_imsi = DataFrame()
        df_rdb_imsi_range = self.fetch_imsi_data_from_rdb()
        for Name, RangeStart, RangeEnd, Allocation in df_rdb_imsi_range.values:
            df_rdb_imsi = df_rdb_imsi.append(DataFrame({'SPONSOR':Name,
                                  'COMMENT':Allocation}, pd.Index(range(RangeStart, RangeEnd), name='IMSI'))) 
        df_rdb_imsi['IMSI'] = df_rdb_imsi.index
        list_agg_imsi = [('_MIN','min'),('_MAX','max'),('_NUM','nunique')]
        list_agg_comment = [('','unique')]
        list_col = ['SPONSOR', df_rdb_imsi.IMSI // 10 ** imsi_agg_precision]
        df_rdb_imsi_group = df_rdb_imsi.groupby(list_col)['IMSI','COMMENT'].agg({'IMSI': list_agg_imsi,
                                                                                 'COMMENT':list_agg_comment}).reset_index()
        df_rdb_imsi_group.columns = [''.join(col) for col in df_rdb_imsi_group.columns]
        df_rdb_imsi_group = pd.concat([df_rdb_imsi_group, DataFrame(df_rdb_imsi_group.COMMENT.tolist())], axis=1)
        df_rdb_imsi_group.drop('COMMENT', axis=1, inplace=True)

        return df_rdb_imsi_group
    
    def msisdn_group_ranges(self):
        
        df_rdb_msisdn = DataFrame()
        df_rdb_msisdn_range = self.fetch_msisdn_data_from_rdb()
        for Name, RangeStart, RangeEnd, Description in df_rdb_msisdn_range.values:
            df_rdb_msisdn = df_rdb_msisdn.append(DataFrame({'SPONSOR':Name,
                                  'COMMENT':Description}, pd.Index(range(RangeStart, RangeEnd), name='MSISDN'))) 
        df_rdb_msisdn['MSISDN'] = df_rdb_msisdn.index
        list_agg_msisdn = [('_MIN','min'),('_MAX','max'),('_NUM','nunique')]
        list_agg_comment = [('','unique')]
        list_col = ['SPONSOR', df_rdb_msisdn.MSISDN // 10 ** msisdn_agg_precision]
        df_rdb_msisdn_group = df_rdb_msisdn.groupby(list_col)['MSISDN','COMMENT'].agg({'MSISDN': list_agg_msisdn,
                                                                                 'COMMENT':list_agg_comment}).reset_index()
        df_rdb_msisdn_group.columns = [''.join(col) for col in df_rdb_msisdn_group.columns]
        df_rdb_msisdn_group = pd.concat([df_rdb_msisdn_group, DataFrame(df_rdb_msisdn_group.COMMENT.tolist())], axis=1)
        df_rdb_msisdn_group.drop('COMMENT', axis=1, inplace=True)

        return df_rdb_msisdn_group
    
#######################################################################################
# Define variables here (Part 1/3)
#######################################################################################

imsi_agg_precision = 3 # Aggregation presision for IMSI rangaes
msisdn_agg_precision = 3 # Aggregation presision for MSISDN rangaes

range_type = 'dmi_external_hlr' # range_type: esim, own_hlr, dmi_external_hlr, test

# sponsor_name: 'S1' / 'S2' / 'S3'...
                
s1_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S1',
                                                 imsi_range_esim = '425019614',
                                                 imsi_range_own_hlr = '425019613',
                                                 imsi_range_dmi_external_hlr = '42501962',
                                                 imsi_range_test = '42501961400',
                                                 msisdn_range = '972540',
                                                 rdb_sponsor_name = 'Partner')

s2_defined_sponsor_ranges = DefinedSponsorRanges(sponsor_name = 'S2',
                                                 imsi_range_esim = '260060145',
                                                 imsi_range_own_hlr = '26006014',
                                                 imsi_range_dmi_external_hlr = '26006014',
                                                 imsi_range_test = '26006014500',
                                                 msisdn_range = '487912',
                                                 rdb_sponsor_name = 'P4 Sp z o o')

# range_type: esim, own_hlr, dmi_external_hlr, test

s1_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s1_defined_sponsor_ranges,
                                  range_type = range_type)

s2_sponsor_ranges = SponsorRanges(defined_sponsor_ranges = s2_defined_sponsor_ranges,
                                  range_type = range_type)


sponsor_ranges = [s1_sponsor_ranges, s2_sponsor_ranges]

#######################################################################################

#######################################################################################

Wall time: 39.1 s


In [20]:
# Under Development

for sponsor_range in sponsor_ranges:
    display(sponsor_range.imsi_group_ranges.loc[sponsor_range.imsi_group_ranges[1].isna()].head(10))

,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1,2,3,4,5,6,7,8
10,S1,425019614010,425019614010000,425019614010999,1000,eSim,None,None,None,None,None,None,None,None
11,S1,425019614011,425019614011000,425019614011999,1000,eSim,None,None,None,None,None,None,None,None
12,S1,425019614012,425019614012000,425019614012999,1000,eSim,None,None,None,None,None,None,None,None
13,S1,425019614013,425019614013000,425019614013999,1000,eSim,None,None,None,None,None,None,None,None
14,S1,425019614014,425019614014000,425019614014999,1000,eSim,None,None,None,None,None,None,None,None
15,S1,425019614015,425019614015000,425019614015999,1000,eSim,None,None,None,None,None,None,None,None
16,S1,425019614016,425019614016000,425019614016999,1000,eSim,None,None,None,None,None,None,None,None
17,S1,425019614017,425019614017000,425019614017999,1000,eSim,None,None,None,None,None,None,None,None
18,S1,425019614018,425019614018000,425019614018999,1000,eSim,None,None,None,None,None,None,None,None
19,S1,425019614019,425019614019000,425019614019999,1000,eSim,None,None,None,None,None,None,None,None


,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1,2,3,4,5,6,7
10,S2,260060145010,260060145010000,260060145010999,1000,eSim,None,None,None,None,None,None,None
11,S2,260060145011,260060145011000,260060145011999,1000,eSim,None,None,None,None,None,None,None
12,S2,260060145012,260060145012000,260060145012999,1000,eSim,None,None,None,None,None,None,None
13,S2,260060145013,260060145013000,260060145013999,1000,eSim,None,None,None,None,None,None,None
14,S2,260060145014,260060145014000,260060145014999,1000,eSim,None,None,None,None,None,None,None
15,S2,260060145015,260060145015000,260060145015999,1000,eSim,None,None,None,None,None,None,None
16,S2,260060145016,260060145016000,260060145016999,1000,eSim,None,None,None,None,None,None,None
17,S2,260060145017,260060145017000,260060145017999,1000,eSim,None,None,None,None,None,None,None
18,S2,260060145018,260060145018000,260060145018999,1000,eSim,None,None,None,None,None,None,None
19,S2,260060145019,260060145019000,260060145019999,1000,eSim,None,None,None,None,None,None,None


In [21]:
# Under Development

for sponsor_range in sponsor_ranges:
    display(sponsor_range.msisdn_group_ranges.loc[sponsor_range.msisdn_group_ranges[1].isna()].head(10))

,SPONSOR,MSISDN,MSISDN_MIN,MSISDN_MAX,MSISDN_NUM,0,1
0,S1,972540000,972540000000,972540000999,1000,Main Range,None
1,S1,972540001,972540001000,972540001999,1000,Main Range,None
2,S1,972540002,972540002000,972540002999,1000,Main Range,None
3,S1,972540003,972540003000,972540003999,1000,Main Range,None
4,S1,972540004,972540004000,972540004999,1000,Main Range,None
5,S1,972540005,972540005000,972540005999,1000,Main Range,None
6,S1,972540006,972540006000,972540006999,1000,Main Range,None
7,S1,972540007,972540007000,972540007999,1000,Main Range,None
8,S1,972540008,972540008000,972540008999,1000,Main Range,None
9,S1,972540009,972540009000,972540009999,1000,Main Range,None


,SPONSOR,MSISDN,MSISDN_MIN,MSISDN_MAX,MSISDN_NUM,0,1,2,3
71,S2,48791271,48791271000,48791271999,1000,Main Range,None,None,None
72,S2,48791272,48791272000,48791272999,1000,Main Range,None,None,None
73,S2,48791273,48791273000,48791273999,1000,Main Range,None,None,None
74,S2,48791274,48791274000,48791274999,1000,Main Range,None,None,None
75,S2,48791275,48791275000,48791275999,1000,Main Range,None,None,None
76,S2,48791276,48791276000,48791276999,1000,Main Range,None,None,None
77,S2,48791277,48791277000,48791277999,1000,Main Range,None,None,None
78,S2,48791278,48791278000,48791278999,1000,Main Range,None,None,None
79,S2,48791279,48791279000,48791279999,1000,Main Range,None,None,None
80,S2,48791280,48791280000,48791280999,1000,Main Range,None,None,None


In [26]:
# Записать выбранные диапазоны IMSI
# Проверить выбранные IMSI на принадлежность к "Roamability Invertory"

# После переноса IMSI в аккаунты Кастомеров проверить выбранные IMSI и диапазоны
# (предварительно перезагрузив данные из OCS)

imsi_range_object = namedtuple('ImsiRange', 'sponsor imsi_range_start imsi_range_end')

### Set the IMSI ranges here ###

s1_imsi_range = imsi_range_object('S1', '425019628009900', '425019628109999')
s2_imsi_range = imsi_range_object('S2', '260060143140000', '260060143149999')

#################################

imsi_ranges = [s1_imsi_range, s2_imsi_range]

for imsi_range in imsi_ranges:
    if imsi_range.imsi_range_start:
        check_dmi_imsi(imsi_range.imsi_range_start, imsi_range.imsi_range_end)
        check_ocs_imsi(imsi_range.imsi_range_start, imsi_range.imsi_range_end)

Checking DMI s_imsi for: 425019628009900 - 425019628109999


,IMSI_RANGE,NUM
0,4250196280,90070


Checking DMI subscriber for: 425019628009900 - 425019628109999
No items
Checking OCS multi_imsi for: 425019628009900 - 425019628109999
No items
Checking OCS SUBSCRIBER_IMSIS for: 425019628009900 - 425019628109999
No items
Checking DMI s_imsi for: 260060143140000 - 260060143149999
No items
Checking DMI subscriber for: 260060143140000 - 260060143149999
No items
Checking OCS multi_imsi for: 260060143140000 - 260060143149999
No items
Checking OCS SUBSCRIBER_IMSIS for: 260060143140000 - 260060143149999
No items


In [25]:
# Проверить запись IMSI в RoamDb

for sponsor_range in sponsor_ranges:
    for imsi_range in imsi_ranges:
        if imsi_range.sponsor == sponsor_range.defined_sponsor_ranges.sponsor_name:
            display(sponsor_range.imsi_group_ranges.loc[
                (sponsor_range.imsi_group_ranges.IMSI_MIN >= int(imsi_range.imsi_range_start)) &
                (sponsor_range.imsi_group_ranges.IMSI_MIN <= int(imsi_range.imsi_range_end))].head(10))

,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1,2
8100,S1,425019628100,425019628100000,425019628100999,1000,DMI,X2One 200129,None
8101,S1,425019628101,425019628101000,425019628101999,1000,DMI,X2One 200129,None
8102,S1,425019628102,425019628102000,425019628102999,1000,DMI,X2One 200129,None
8103,S1,425019628103,425019628103000,425019628103999,1000,DMI,X2One 200129,None
8104,S1,425019628104,425019628104000,425019628104999,1000,DMI,X2One 200129,None
8105,S1,425019628105,425019628105000,425019628105999,1000,DMI,X2One_200122_5K_Dialog,None
8106,S1,425019628106,425019628106000,425019628106999,1000,DMI,X2One_200122_5K_Dialog,None
8107,S1,425019628107,425019628107000,425019628107999,1000,DMI,X2One_200122_5K_Dialog,None
8108,S1,425019628108,425019628108000,425019628108999,1000,DMI,X2One_200122_5K_Dialog,None
8109,S1,425019628109,425019628109000,425019628109999,1000,DMI,X2One_200122_5K_Dialog,None


,SPONSOR,IMSI,IMSI_MIN,IMSI_MAX,IMSI_NUM,0,1,2,3,4,5,6,7,8
3140,S2,260060143140,260060143140000,260060143140999,1000,DMI,X2One 200129,None,None,None,None,None,None,None
3141,S2,260060143141,260060143141000,260060143141999,1000,DMI,X2One 200129,None,None,None,None,None,None,None
3142,S2,260060143142,260060143142000,260060143142999,1000,DMI,X2One 200129,None,None,None,None,None,None,None
3143,S2,260060143143,260060143143000,260060143143999,1000,DMI,X2One 200129,None,None,None,None,None,None,None
3144,S2,260060143144,260060143144000,260060143144999,1000,DMI,X2One 200129,None,None,None,None,None,None,None
3145,S2,260060143145,260060143145000,260060143145999,1000,DMI,X2One_200122_5K_Dialog,None,None,None,None,None,None,None
3146,S2,260060143146,260060143146000,260060143146999,1000,DMI,X2One_200122_5K_Dialog,None,None,None,None,None,None,None
3147,S2,260060143147,260060143147000,260060143147999,1000,DMI,X2One_200122_5K_Dialog,None,None,None,None,None,None,None
3148,S2,260060143148,260060143148000,260060143148999,1000,DMI,X2One_200122_5K_Dialog,None,None,None,None,None,None,None
3149,S2,260060143149,260060143149000,260060143149999,1000,DMI,X2One_200122_5K_Dialog,None,None,None,None,None,None,None


### ICCID allocation

In [41]:
sql_srt="""
SELECT 
o.Name,siccid.RangeStart,siccid.RangeEnd,siccid.Allocation
FROM [dbo].[SponsorICCID] siccid
LEFT JOIN [dbo].[Sponsors] s ON siccid.SponsorID = s.ID
LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
WHERE o.Name LIKE 'X2%';"""
with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
    df_iccid = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
df_iccid.head()

df_iccid['Count'] = df_iccid.RangeEnd - df_iccid.RangeStart + 1
dict_agg = {'RangeStart':'min', 'RangeEnd':'max', 'Count':'sum'}
list_col = ['Name', 'Allocation', df_iccid.RangeStart.astype('str').str.slice(0,12)]
df_iccid.groupby(list_col, as_index=False).agg(dict_agg).sort_values(by=['RangeStart','RangeEnd'])

,Name,Allocation,RangeStart,RangeEnd,Count
1,X2one,ALL 190826,8997212330000000000,8997212330099999999,100000000
6,X2one,Dailoq 500 190830,8997212330099000000,8997212330099000499,500
0,X2one,1K DIALOQ #3501982,8997212330099000500,8997212330099001499,1000
9,X2one,Reserved for DENT 8.5K,8997212330099001500,8997212330099009999,8500
7,X2one,Dialoq 5 test eSim,8997212330099010000,8997212330099010004,5
8,X2one,For Tests,8997212330099010000,8997212330099010999,1000
2,X2one,DENT 191122,8997212330099010005,8997212330099010206,202
3,X2one,DENT 191127,8997212330099010207,8997212330099010706,500
10,X2one,Tcom Test 200104,8997212330099010707,8997212330099010716,10
4,X2one,DENT 200114,8997212330099010717,8997212330099011216,500


In [68]:
the_last = 8997212330099039199
num = 5000
print(f'{the_last + 1} - {the_last + num}')
print(f"BETWEEN '{the_last + 1}' AND '{the_last + num}'")

8997212330099039200 - 8997212330099044199
BETWEEN '8997212330099039200' AND '8997212330099044199'


### Developments

In [3]:
%%time
sponsor_imsi_range = '260060140'  # All range DMI

agg_precision = 12

sql_srt="""
SELECT
o.Name,i.RangeStart,i.RangeEnd,i.Allocation
FROM ImsiRange i
LEFT JOIN [dbo].[Sponsors] s ON i.SponsorID = s.ID
LEFT JOIN [dbo].[Operators] o ON o.ID = s.OperatorID
WHERE o.Name LIKE 'P4%'
AND i.RangeStart LIKE '{0}%';""".format(sponsor_imsi_range)
with rb.MssqlConnect('172.19.11.59', 'RoamDb', 'roamdb', 'roamdbpasswd') as cnxn:
    df_rdb_imsi_range = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

df_rdb_imsi_range.head(3)

Wall time: 1.07 s


,Name,RangeStart,RangeEnd,Allocation
0,P4 Sp z o o,260060140000000,260060149999999,DMI
1,P4 Sp z o o,260060140900000,260060140999999,DMI for Telzar SIM's
2,P4 Sp z o o,260060140000000,260060140999999,DMI Allocated


In [4]:
%%time
df_rdb_imsi = DataFrame()
for Name, RangeStart, RangeEnd, Allocation in df_rdb_imsi_range.values:
    df_rdb_imsi = df_rdb_imsi.append(DataFrame({'SPONSOR':Name,
                                  'IMSI':range(RangeStart, RangeEnd),
                                  'COMMENT':Allocation}))

Wall time: 2.33 s


In [5]:
%%time
df_rdb_imsi = DataFrame()
for Name, RangeStart, RangeEnd, Allocation in df_rdb_imsi_range.values:
    df_rdb_imsi = df_rdb_imsi.append(DataFrame({'SPONSOR':Name,
                                  'COMMENT':Allocation}, pd.Index(range(RangeStart, RangeEnd), name='IMSI')))
df_rdb_imsi['IMSI'] = df_rdb_imsi.index
df_rdb_imsi.head()

Wall time: 1.07 s


,SPONSOR,COMMENT,IMSI
IMSI,,,
260060140000000,P4 Sp z o o,DMI,260060140000000
260060140000001,P4 Sp z o o,DMI,260060140000001
260060140000002,P4 Sp z o o,DMI,260060140000002
260060140000003,P4 Sp z o o,DMI,260060140000003
260060140000004,P4 Sp z o o,DMI,260060140000004


In [8]:
%%time
list_agg_imsi = [('_MIN','min'),('_MAX','max'),('_NUM','nunique')]
list_agg_comment = [('','unique')]
list_col = ['SPONSOR', df_rdb_imsi.IMSI.astype('str').str.slice(0, agg_precision)]
df_rdb_imsi_group = df_rdb_imsi.groupby(list_col)['IMSI','COMMENT'].agg({'IMSI': list_agg_imsi,
                                                                         'COMMENT':list_agg_comment}).reset_index()
df_rdb_imsi_group.head()

Wall time: 18.3 s


SPONSOR          IMSI                                          \
                                         _MIN             _MAX  _NUM   
0  P4 Sp z o o  260060140000  260060140000000  260060140000999  1000   
1  P4 Sp z o o  260060140001  260060140001000  260060140001999  1000   
2  P4 Sp z o o  260060140002  260060140002000  260060140002999  1000   
3  P4 Sp z o o  260060140003  260060140003000  260060140003999  1000   
4  P4 Sp z o o  260060140004  260060140004000  260060140004999  1000   

                COMMENT  
                         
0  [DMI, DMI Allocated]  
1  [DMI, DMI Allocated]  
2  [DMI, DMI Allocated]  
3  [DMI, DMI Allocated]  
4  [DMI, DMI Allocated]

In [13]:
%%time
list_agg_imsi = [('_MIN','min'),('_MAX','max'),('_NUM','nunique')]
list_agg_comment = [('','unique')]
list_col = ['SPONSOR', df_rdb_imsi.IMSI // 1000]
df_rdb_imsi_group = df_rdb_imsi.groupby(list_col)['IMSI','COMMENT'].agg({'IMSI': list_agg_imsi,
                                                                         'COMMENT':list_agg_comment}).reset_index()
df_rdb_imsi_group.head()

Wall time: 6.95 s


SPONSOR          IMSI                                          \
                                         _MIN             _MAX  _NUM   
0  P4 Sp z o o  260060140000  260060140000000  260060140000999  1000   
1  P4 Sp z o o  260060140001  260060140001000  260060140001999  1000   
2  P4 Sp z o o  260060140002  260060140002000  260060140002999  1000   
3  P4 Sp z o o  260060140003  260060140003000  260060140003999  1000   
4  P4 Sp z o o  260060140004  260060140004000  260060140004999  1000   

                COMMENT  
                         
0  [DMI, DMI Allocated]  
1  [DMI, DMI Allocated]  
2  [DMI, DMI Allocated]  
3  [DMI, DMI Allocated]  
4  [DMI, DMI Allocated]